The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$. To run these calculations, use the modified version of sage from
https://github.com/bodnalev/sage

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices partitioned 
into 3 parts using an additional 2-ary relation indicating different parts (called CTGp)

2. The second cell performs the basic calculation of upper bounding edges 
in the theory for Proposition 3.1. It gives the $\alpha_{3, 1}$ = a31 = 126373441/504000000 ~= 0.250740954365079... upper bound.
The certificate proving the claim is saved to the file "proposition_3_1.pickle".

3. The next cell lower bounds the max-cut ration (k222) for Proposition 3.2.
Note at the optimal construction this value is exactly 25/121 ~= 0.19834710743801652892...
The exact lower bound provided is $\alpha_{3, 2}$ = a32 = 1607168566087/8282009829376 ~= 0.194055380179148...
The certificate proving the claim is saved to the file "proposition_3_2.pickle".

5. The following cell works in the 3 partitioned theory.
It uses the lower bound b33=0.19 on the edges in the correct parts, where $\beta_{3, 3}$ = b33 < a32 from the previous calculation.
The calculations provide the precise density bound that there are less bad
edges than missing edges asymptotically on the top
level. Here bad and missing is defined compared to the expected construction.
The certificate proving this claim is saved to the file "proposition_3_3.pickle".

6. The next cell establishes the theory for 3 colored graphs appearing as
a link of a vertex. The patterns excluded all would result in a $C_5^-$
assuming that all the 3-edges between the three parts are present.

7. Then the calculation on the link graph is performed.
The calculations provide the precise density bound that there are less bad
edges than missing edges asymptotically. The
certificate proving this claim is saved to the file "proposition_3_4.pickle".

8. The first 6 cells perform the entire calculation from scratch. If one
only wants to verify that any of the certificates are indeed correct, it is enough
to run the corresponding cell from the final 4. For each step above, it loads the generated
certificates and verifies that the matrices are indeed positive semidefinite
and that the bound they prove is exactly as claimed.

In [3]:
### Theory for 3-graphs, with k4m and c5m excluded
k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory


# Graphs representing a three-partition
P = Theory("3Partition", relation_name="part", arity=2, is_ordered=False)
P.exclude([P(3, part=[[0, 1]]), P(4, part=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]])])
ThreePartitionedThreeGraphTheory = combine("3PartitionNoC5m", TGp, P)
CTGp = ThreePartitionedThreeGraphTheory

{'arity': 3, 'ordered': False, 'group': 0} {'arity': 2, 'ordered': False, 'group': 0} edges


In [4]:
### This part just gives a standard upper bound on the number of edges without C5- and K4- (Proposition 3.1)

a31 = TGp.optimize(TGp(3, edges=[[0, 1, 2]]), 7,  exact=True, denom=1024*3125, file="certificates/proposition_3_1")
print("The initial upper bound on the Turan density from Proposition 3.1 is {} ~= {}".format(a31, a31.n()))

Base flags generated, their number is 1127
The relevant ftypes are constructed, their number is 12
Block sizes before symmetric/asymmetric change is applied: [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28]


Done with mult table for Ftype on 5 points with edges=(012 023 024 134): : 12it [00:02,  4.68it/s]


Tables finished
Constraints finished
Running sdp without construction. Used block sizes are [5, 26, 48, 11, 21, 14, 374, 31, 147, 35, 48, 23, 74, 10, 24, 28, 17, 27, 27, 12, 19, 8, 20, -1127, -2]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 4.75e-01 Pobj: -7.8579696e+01 Ad: 1.33e-01 Dobj:  1.1544783e+00 
Iter:  2 Ap: 6.77e-01 Pobj: -2.0704711e+02 Ad: 5.29e-01 Dobj:  4.5238754e+00 
Iter:  3 Ap: 8.72e-01 Pobj: -4.1716308e+02 Ad: 6.11e-01 Dobj:  8.5665561e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -4.5342396e+02 Ad: 8.79e-01 Dobj:  7.8308318e-03 
Iter:  5 Ap: 1.00e+00 Pobj: -4.0202345e+02 Ad: 8.94e-01 Dobj: -1.4867891e-01 
Iter:  6 Ap: 7.60e-01 Pobj: -3.7223032e+02 Ad: 8.53e-01 Dobj: -1.7578573e-01 
Iter:  7 Ap: 5.16e-01 Pobj: -3.5064919e+02 Ad: 6.62e-01 Dobj: -1.8586373e-01 
Iter:  8 Ap: 1.23e-01 Pobj: -3.5446817e+02 Ad: 4.18e-01 Dobj: -1.7747874e-01 
Iter:  9 Ap: 6.38e-01 Pobj: -3.2583290e+02 Ad: 4.21e-01 Dobj: -1.7502293e-01 
Iter: 10 Ap: 

100%|███████████████████████████████████████████| 23/23 [00:14<00:00,  1.57it/s]


Calculating resulting bound


100%|███████████████████████████████████████████| 12/12 [06:07<00:00, 30.62s/it]


The rounded result is -126373441/504000000
The initial upper bound on the Turan density from Proposition 3.1 is 126373441/504000000 ~= 0.250740954365079


In [5]:
### This code that minimizes the max-cut ratio K222 (Proposition 3.2)

# beta_{3, 2} constant
b32 = 1/4 - 1/100000
edge = TGp(3, edges=[[0, 1, 2]])

# First the typed f222 is constructed
f222 = TGp.pattern(6, ftype=[0, 1, 2], edges=[[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]).afae()
# Then k222 is the projection of f222, this takes care of the automorphisms
k222 = f222.project()

gamma = TGp.optimize(k222, 7, maximize=False, positives=[edge - b32], exact=True, denom=1024*1024, file="certificates/proposition_3_2")
a32 = gamma / a31

print("The max-cut ratio returned by Proposition 3.2 is at least {} ~= {}".format(a32, a32.n()))

Base flags generated, their number is 1127
The relevant ftypes are constructed, their number is 12
Block sizes before symmetric/asymmetric change is applied: [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28]


Done with mult table for Ftype on 5 points with edges=(012 023 024 134): : 12it [00:02,  5.10it/s]


Tables finished


Done with positivity constraint 0: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


Constraints finished
Running sdp without construction. Used block sizes are [5, 26, 48, 11, 21, 14, 374, 31, 147, 35, 48, 23, 74, 10, 24, 28, 17, 27, 27, 12, 19, 8, 20, -1127, -5]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 6.09e-01 Pobj: -9.2052112e+01 Ad: 1.47e-01 Dobj:  3.1143968e+00 
Iter:  2 Ap: 7.13e-01 Pobj: -2.0415132e+02 Ad: 5.43e-01 Dobj:  3.0456394e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -3.7441986e+02 Ad: 7.10e-01 Dobj:  6.9577767e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -3.7587362e+02 Ad: 8.98e-01 Dobj:  1.2288131e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -3.0805356e+02 Ad: 8.97e-01 Dobj:  3.8922659e-02 
Iter:  6 Ap: 7.42e-01 Pobj: -2.7570722e+02 Ad: 7.38e-01 Dobj:  2.5778021e-02 
Iter:  7 Ap: 4.99e-01 Pobj: -2.1738401e+02 Ad: 4.00e-01 Dobj:  2.9783858e-02 
Iter:  8 Ap: 8.73e-01 Pobj: -1.4405554e+02 Ad: 5.91e-01 Dobj:  3.7240210e-02 
Iter:  9 Ap: 1.00e+00 Pobj: -1.1671799e+02 Ad: 5.77e-01 Dobj:  4.1653897e-02 
Iter: 10 Ap: 6.03e-01 Pobj: -

100%|███████████████████████████████████████████| 23/23 [00:11<00:00,  1.96it/s]


Calculating resulting bound


100%|███████████████████████████████████████████| 12/12 [06:02<00:00, 30.21s/it]


The rounded result is 229595509441/4718592000000
The max-cut ratio returned by Proposition 3.2 is at least 1607168566087/8282009829376 ~= 0.194055380179148


In [6]:
### This is the code that performs the calculations on the theory with three partition (Proposition 3.3)

# edge with (C)orrect partition
C = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]])
# edge with (C)orrect partition (p)ointed
Cp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]], ftype=[0])

# edge with (B)ad partition
B = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]])

# edge with (B)ad partition (p)ointed 
Bp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]], ftype=[0])

# (M)issing edge with good partition
M = CTGp(3, edges=[], part=[[0, 1], [0, 2], [1, 2]])

# positivity assumptions:
# each point, good edges are more than bad edges (divided by two due to the symmetry between the parts)
# edge density is larger than b33 (which is smaller than a32)
b33 = 19/100
assums = [Cp - Bp/2, C - b33]

# optimal construction and its derivatives
symbolic_constr = CTGp.blowup_construction(6, 3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]], symbolic_parts=True).set_sum()
ders = symbolic_constr.derivatives([1/3, 1/3])

# bad is less than missing, proven by (B)ad (M)inum (M)issing is at most 0.
CTGp.optimize(B - M*(99/100), 6, positives=assums, exact=True, construction=ders, denom=2048, file="certificates/proposition_3_3")

100%|██████████████████████████████████████████| 28/28 [00:00<00:00, 145.18it/s]


Base flags generated, their number is 2840
The relevant ftypes are constructed, their number is 20
Block sizes before symmetric/asymmetric change is applied: [35, 57, 82, 123, 123, 123, 40, 60, 60, 60, 60, 60, 20, 30, 30, 30, 30, 30, 30, 30]


Done with mult table for Ftype on 4 points with edges=(023 123), part=(01 02 03 12 13): : 20it [00:01, 10.21it/s]


Tables finished


Done with positivity constraint 1: 100%|██████████| 2/2 [00:02<00:00,  1.40s/it]


Constraints finished
Adjusting table with kernels from construction
Running SDP after kernel correction. Used block sizes are [23, 9, 30, 23, 12, 68, 33, 87, 29, 91, 43, 80, 14, 26, 39, 21, 21, 39, 39, 21, 33, 27, 39, 21, 12, 8, 24, 6, 25, 5, 18, 12, 18, 12, 12, 15, 30, 13, 17, -2840, -50]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 3.71e-01 Pobj: -3.3468669e+01 Ad: 1.55e-01 Dobj:  7.8728718e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -9.5892055e+01 Ad: 5.78e-01 Dobj:  5.3611359e+01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0289402e+02 Ad: 8.54e-01 Dobj:  8.8250175e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0881544e+02 Ad: 9.25e-01 Dobj:  7.1800267e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.2001459e+02 Ad: 8.71e-01 Dobj:  9.0959437e-02 
Iter:  6 Ap: 1.00e+00 Pobj: -1.4453093e+02 Ad: 6.37e-01 Dobj:  5.3542728e-02 
Iter:  7 Ap: 4.75e-01 Pobj: -1.4249146e+02 Ad: 6.28e-01 Dobj:  4.0173256e-02 
Iter:  8 Ap: 4.74e-01 Pobj: -1.3607610e+02 Ad: 5.20e-01 Dobj:  2.7025547e-

100%|███████████████████████████████████████████| 20/20 [05:41<00:00, 17.08s/it]


Final rounded bound is 0


0

In [7]:
### Theory for 3-colored 2-graphs, the colors are not interchangeable here
Asymm3ColorTheory = combine("Asymm3Colors", Color0, Color1, Color2, symmetries=NoSymmetry)
# Again, force the colors to be disjoint
Asymm3ColorTheory.exclude([
    Asymm3ColorTheory(1), 
    Asymm3ColorTheory(1, C0=[0], C1=[0]), 
    Asymm3ColorTheory(1, C0=[0], C2=[0]), 
    Asymm3ColorTheory(1, C1=[0], C2=[0]),
    Asymm3ColorTheory(1, C0=[0], C1=[0], C2=[0])
])
ColoredLinkGraphTheory = combine("ColoredLinkGraph", GraphTheory, Asymm3ColorTheory)
CLGT = ColoredLinkGraphTheory
# These non-induced patterns guarantee the theory represents a 3 colored c5m-free 3-graph's link.
CLGT.exclude([
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[1, 2], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[1, 2], C1=[0], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[3], C2=[1, 2]),
    
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[1], C1=[0], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[2], C2=[1]),

    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[2], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[0, 1], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[0, 1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[], C2=[0, 1]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[2], C2=[0, 1])
])

In [8]:
### Vertex stability part (Proposition 3.4)

edge_00 = CLGT(2, edges=[[0, 1]], C0=[0, 1], C1=[], C2=[])
edge_11 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[])
edge_22 = CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
edge_01 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[])
edge_12 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0], C2=[1])
edge_02 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1])

point0 = CLGT(1, C0 = [0])
point1 = CLGT(1, C1 = [0])
point2 = CLGT(1, C2 = [0])

positives = [
    edge_12 - edge_01, 
    edge_12 - edge_02,
    edge_01 + edge_02 + edge_12 - 1/8,
    point0 - 1/4, 
    point1 - 1/4, 
    point2 - 1/4
]

# (M)issing edge with good colors
M = CLGT(2, edges=[], C0=[], C1=[0], C2=[1]) 

# edges with (B)ad colors 
B = sum([
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
])

CLGT.optimize(B - M*9/10, 5, positives = positives, exact=True, file="certificates/proposition_3_4")

Base flags generated, their number is 450
The relevant ftypes are constructed, their number is 43
Block sizes before symmetric/asymmetric change is applied: [28, 28, 28, 24, 15, 15, 24, 15, 12, 15, 15, 15, 24, 12, 8, 9, 9, 8, 12, 8, 9, 6, 6, 6, 8, 9, 9, 8, 9, 8, 12, 10, 7, 7, 10, 7, 7, 7, 7, 10, 10, 10, 10]


Done with mult table for Ftype on 3 points with edges=(01 02 12), C0=(0 1 2), C1=(), C2=(): : 43it [00:17,  2.47it/s]


Tables finished


Done with positivity constraint 5: 100%|██████████| 6/6 [00:00<00:00,  9.50it/s]


Constraints finished
Running sdp without construction. Used block sizes are [28, 28, 28, 12, 12, 12, 3, 12, 3, 12, 12, 12, 3, 12, 12, 3, 12, 3, 12, 3, 12, 12, 10, 2, 8, 8, 1, 8, 1, 8, 10, 2, 8, 8, 1, 6, 6, 6, 8, 8, 1, 8, 1, 8, 8, 1, 8, 10, 2, 8, 2, 6, 1, 6, 1, 8, 2, 6, 1, 6, 1, 6, 1, 6, 1, 8, 2, 6, 4, 6, 4, 6, 4, -450, -527]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -4.9907708e+01 Ad: 7.03e-01 Dobj:  7.4258520e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.1781514e+01 Ad: 9.53e-01 Dobj: -1.1683632e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -5.2769043e+01 Ad: 9.47e-01 Dobj: -7.8346776e-02 
Iter:  4 Ap: 8.94e-01 Pobj: -4.5884511e+01 Ad: 8.45e-01 Dobj: -5.1164934e-02 
Iter:  5 Ap: 9.01e-01 Pobj: -2.6397013e+01 Ad: 5.30e-01 Dobj: -1.0938337e-02 
Iter:  6 Ap: 3.06e-01 Pobj: -2.4483384e+01 Ad: 4.85e-01 Dobj:  2.0486501e-02 
Iter:  7 Ap: 9.52e-01 Pobj: -1.4017574e+01 Ad: 5.78e-01 Dobj:  4.4050725e-02 
Iter:  8 Ap: 1.00e+00 Pobj: -7.5147141

100%|██████████████████████████████████████████| 73/73 [00:00<00:00, 362.53it/s]


Calculating resulting bound


100%|███████████████████████████████████████████| 43/43 [00:07<00:00,  5.49it/s]

The rounded result is 0


0

Verify the certificates produced
===============================

Can be run without running the above cells. Note however that the 
initial call to these cells might take longer, due to the calculation of
the multiplication tables. Once that is complete and stored, these cells 
verify the results fairly quickly.

In [9]:
### Verify Proposition 3.1

k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory
TGp.verify("certificates/proposition_3_1", TGp(3, edges=[[0, 1, 2]]), 7)

Checking X matrices


12it [04:12, 21.04s/it]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


12it [00:02,  5.24it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


12it [05:21, 26.76s/it]


The solution is valid, it proves the bound 126373441/504000000


126373441/504000000

In [10]:
### Verify Proposition 3.2

k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory

b32 = 1/4 - 1/100000
edge = TGp(3, edges=[[0, 1, 2]])
f222 = TGp.pattern(6, ftype=[0, 1, 2], edges=[[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]).afae()
k222 = f222.project()

TGp.verify("certificates/proposition_3_2", k222, 7, maximize=False, positives=[edge - b32])

Checking X matrices


12it [03:20, 16.68s/it]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


12it [00:02,  5.01it/s]


Done with positivity constraint 0
Done calculating linear constraints
Calculating the bound provided by the certificate


12it [05:24, 27.07s/it]


The solution is valid, it proves the bound 229595509441/4718592000000


229595509441/4718592000000

In [11]:
### Verify Proposition 3.3

k4m = ThreeGraphTheory.pattern(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
c5m = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
ThreeGraphTheory.exclude([k4m, c5m])
TGp = ThreeGraphTheory
P = Theory("3Partition", relation_name="part", arity=2, is_ordered=False)
P.exclude([P(3, part=[[0, 1]]), P(4, part=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]])])
ThreePartitionedThreeGraphTheory = combine("3PartitionNoC5m", TGp, P)
CTGp = ThreePartitionedThreeGraphTheory

C = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]])
Cp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [0, 2], [1, 2]], ftype=[0])
B = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]])
Bp = CTGp(3, edges=[[0, 1, 2]], part=[[0, 1], [1, 2]], ftype=[0])
M = CTGp(3, edges=[], part=[[0, 1], [0, 2], [1, 2]])
b33 = 19/100
assums = [Cp - Bp/2, C - b33]
CTGp.verify("certificates/proposition_3_3", B + (-99/100)*M, 6, positives=assums)

{'arity': 3, 'ordered': False, 'group': 0} {'arity': 2, 'ordered': False, 'group': 0} edges
Checking X matrices


20it [00:05,  3.60it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


20it [00:01, 16.06it/s]


Done with positivity constraint 0
Done with positivity constraint 1
Done calculating linear constraints
Calculating the bound provided by the certificate


20it [04:53, 14.67s/it]


The solution is valid, it proves the bound 0


0

In [12]:
### Verify Proposition 3.4

Asymm3ColorTheory = combine("Asymm3Colors", Color0, Color1, Color2, symmetries=NoSymmetry)
Asymm3ColorTheory.exclude([
    Asymm3ColorTheory(1), 
    Asymm3ColorTheory(1, C0=[0], C1=[0]), 
    Asymm3ColorTheory(1, C0=[0], C2=[0]), 
    Asymm3ColorTheory(1, C1=[0], C2=[0]),
    Asymm3ColorTheory(1, C0=[0], C1=[0], C2=[0])
])
ColoredLinkGraphTheory = combine("ColoredLinkGraph", GraphTheory, Asymm3ColorTheory)
CLGT = ColoredLinkGraphTheory
CLGT.exclude([
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[1, 2], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[1, 2], C1=[0], C2=[3]),
    CLGT.pattern(4, edges=[[0, 1], [2, 3]], C0=[0], C1=[3], C2=[1, 2]),
    
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[1], C1=[0], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0], C1=[2], C2=[1]),

    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[2], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[0, 1], C1=[], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[0, 1], C2=[]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[0, 1], C2=[2]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[2], C1=[], C2=[0, 1]),
    CLGT.pattern(3, edges=[[0, 1], [1, 2]], C0=[], C1=[2], C2=[0, 1])
])

edge_00 = CLGT(2, edges=[[0, 1]], C0=[0, 1], C1=[], C2=[])
edge_11 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[])
edge_22 = CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
edge_01 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[])
edge_12 = CLGT(2, edges=[[0, 1]], C0=[], C1=[0], C2=[1])
edge_02 = CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1])
point0 = CLGT(1, C0 = [0])
point1 = CLGT(1, C1 = [0])
point2 = CLGT(1, C2 = [0])
positives = [
    edge_12 - edge_01, 
    edge_12 - edge_02,
    edge_01 + edge_02 + edge_12 - 1/8,
    point0 - 1/4, 
    point1 - 1/4, 
    point2 - 1/4
]
M = CLGT(2, edges=[], C0=[], C1=[0], C2=[1]) 
B = sum([
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[0], C1=[], C2=[1]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[0, 1], C2=[]),
    CLGT(2, edges=[[0, 1]], C0=[], C1=[], C2=[0, 1])
])

CLGT.verify("certificates/proposition_3_4", B - M*9/10, 5, positives = positives)

Checking X matrices


43it [00:00, 652.57it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


43it [00:00, 521.30it/s]

Done with positivity constraint 0
Done with positivity constraint 1


Done with positivity constraint 2
Done with positivity constraint 3
Done with positivity constraint 4
Done with positivity constraint 5
Done calculating linear constraints
Calculating the bound provided by the certificate


43it [00:05,  8.41it/s]

The solution is valid, it proves the bound 0


0